In [1]:
import os
import pandas as pd
import numpy as np

import locale
from locale import atof
import xlsxwriter

from utilities.PreProcessingUtil import preprocessing

In [2]:
df = pd.read_excel(
     os.path.join("datasets", "pollutants", "besiktas.xlsx"),
     engine='openpyxl',
     parse_dates=True,
     thousands='.'
)
print (df)

                    Tarih PM10 ( µg/m3 ) SO2 ( µg/m3 ) CO ( µg/m3 )  \
0     2012-01-01 01:00:56              -             -            -   
1     2012-01-01 02:00:56              -             -            -   
2     2012-01-01 03:00:56              -             -            -   
3     2012-01-01 04:00:56              -             -            -   
4     2012-01-01 05:00:56              -             -            -   
...                   ...            ...           ...          ...   
81690 2021-04-26 19:00:56          11,10          1,00       296,70   
81691 2021-04-26 20:00:56          17,40          0,80       254,10   
81692 2021-04-26 21:00:56          30,00          0,70       180,00   
81693 2021-04-26 22:00:56          18,60          1,00       399,00   
81694 2021-04-26 23:00:56              -          1,10       446,60   

      NO2 ( µg/m3 ) NOX ( µg/m3 ) O3 ( µg/m3 ) PM 2.5 ( µg/m3 )  
0                 -             -            -                -  
1              

In [3]:
# Replace '-' string with NaN
df = df.replace ('-', '-1')

# Also fixing ',' delimeter with '.' for float conversion
# . is for thousands , for the last delimeter
df['PM10 ( µg/m3 )'] = df['PM10 ( µg/m3 )'].astype(str).str.replace('.','')
df['PM10 ( µg/m3 )'] = df['PM10 ( µg/m3 )'].astype(str).str.replace(',','.')

# Casting str to float
df['PM10 ( µg/m3 )'] = pd.to_numeric(df['PM10 ( µg/m3 )'], downcast="float")

In [4]:
# Counting number of -1 in the column
df[df == '-1'].count () 

/Users/deneme/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Tarih                   0
PM10 ( µg/m3 )          0
SO2 ( µg/m3 )       61745
CO ( µg/m3 )        62154
NO2 ( µg/m3 )       63773
NOX ( µg/m3 )       67718
O3 ( µg/m3 )        62743
PM 2.5 ( µg/m3 )    67638
dtype: int64

In [5]:
# Filling empty rows
# https://towardsdatascience.com/7-ways-to-handle-missing-values-in-machine-learning-1a6326adf79e
# 1- We cant just delete them because we need consistent timestamps
# 2- If too many empty rows exists we should discard them
# 3- Replacing missing data with mean/median
# 3.1- This does not cover the covariance between features
df['PM10 ( µg/m3 )'] = df['PM10 ( µg/m3 )'].replace (-1.0, df[df != -1]['PM10 ( µg/m3 )'].median ())

In [6]:
df['PM10 ( µg/m3 )']

0        26.1
1        26.1
2        26.1
3        26.1
4        26.1
         ... 
81690    11.1
81691    17.4
81692    30.0
81693    18.6
81694    26.1
Name: PM10 ( µg/m3 ), Length: 81695, dtype: float32

## Town - Pollution Dataset

In [7]:
# creating dataframes per town
dfAksaray = pd.read_excel(os.path.join("datasets", "pollutants", "aksaray.xlsx"),engine='openpyxl',parse_dates=True,thousands='.')
dfBesiktas = pd.read_excel(os.path.join("datasets", "pollutants", "besiktas.xlsx"),engine='openpyxl',parse_dates=True,thousands='.')
dfBuyukada = pd.read_excel(os.path.join("datasets", "pollutants", "buyukada.xlsx"),engine='openpyxl',parse_dates=True,thousands='.')
dfCatladıkapı = pd.read_excel(os.path.join("datasets", "pollutants", "catladıkapı.xlsx"),engine='openpyxl',parse_dates=True,thousands='.')
dfEsenler = pd.read_excel(os.path.join("datasets", "pollutants", "esenler.xlsx"),engine='openpyxl',parse_dates=True,thousands='.')
dfKadıkoy = pd.read_excel(os.path.join("datasets", "pollutants", "kadıkoy.xlsx"),engine='openpyxl',parse_dates=True,thousands='.')
dfKandilli = pd.read_excel(os.path.join("datasets", "pollutants", "kandilli.xlsx"),engine='openpyxl',parse_dates=True,thousands='.')
dfKartal = pd.read_excel(os.path.join("datasets", "pollutants", "kartal.xlsx"),engine='openpyxl',parse_dates=True,thousands='.')
dfMecidiyekoy = pd.read_excel(os.path.join("datasets", "pollutants", "mecidiyekoy.xlsx"),engine='openpyxl',parse_dates=True,thousands='.')
dfUmraniye = pd.read_excel(os.path.join("datasets", "pollutants", "umraniye.xlsx"),engine='openpyxl',parse_dates=True,thousands='.')

In [8]:
import os
import pandas as pd
import numpy as np

def fixValues(df):
    # Replace '-' string with NaN
    df = df.replace ('-', '-1')

    # Also fixing ',' delimeter with '.' for float conversion '.' is for thousands , for the last delimeter
    # Casting str to float
    df['PM10 ( µg/m3 )'] = df['PM10 ( µg/m3 )'].astype(str).str.replace('.','')
    df['PM10 ( µg/m3 )'] = df['PM10 ( µg/m3 )'].astype(str).str.replace(',','.')
    df['PM10 ( µg/m3 )'] = pd.to_numeric(df['PM10 ( µg/m3 )'], downcast="float")
    
    df['SO2 ( µg/m3 )'] = df['SO2 ( µg/m3 )'].astype(str).str.replace('.','')
    df['SO2 ( µg/m3 )'] = df['SO2 ( µg/m3 )'].astype(str).str.replace(',','.')
    df['SO2 ( µg/m3 )'] = pd.to_numeric(df['SO2 ( µg/m3 )'], downcast="float")
    
    df['CO ( µg/m3 )'] = df['CO ( µg/m3 )'].astype(str).str.replace('.','')
    df['CO ( µg/m3 )'] = df['CO ( µg/m3 )'].astype(str).str.replace(',','.')
    df['CO ( µg/m3 )'] = pd.to_numeric(df['CO ( µg/m3 )'], downcast="float")
    
    df['NO2 ( µg/m3 )'] = df['NO2 ( µg/m3 )'].astype(str).str.replace('.','')
    df['NO2 ( µg/m3 )'] = df['NO2 ( µg/m3 )'].astype(str).str.replace(',','.')
    df['NO2 ( µg/m3 )'] = pd.to_numeric(df['NO2 ( µg/m3 )'], downcast="float")
    
    df['NOX ( µg/m3 )'] = df['NOX ( µg/m3 )'].astype(str).str.replace('.','')
    df['NOX ( µg/m3 )'] = df['NOX ( µg/m3 )'].astype(str).str.replace(',','.')
    df['NOX ( µg/m3 )'] = pd.to_numeric(df['NOX ( µg/m3 )'], downcast="float")
    
    df['O3 ( µg/m3 )'] = df['O3 ( µg/m3 )'].astype(str).str.replace('.','')
    df['O3 ( µg/m3 )'] = df['O3 ( µg/m3 )'].astype(str).str.replace(',','.')
    df['O3 ( µg/m3 )'] = pd.to_numeric(df['O3 ( µg/m3 )'], downcast="float")
    
    df['PM 2.5 ( µg/m3 )'] = df['PM 2.5 ( µg/m3 )'].astype(str).str.replace('.','')
    df['PM 2.5 ( µg/m3 )'] = df['PM 2.5 ( µg/m3 )'].astype(str).str.replace(',','.')
    df['PM 2.5 ( µg/m3 )'] = pd.to_numeric(df['PM 2.5 ( µg/m3 )'], downcast="float")
    
    return df
    
def fillEmptyRows(df):
    # Filling empty rows
    # https://towardsdatascience.com/7-ways-to-handle-missing-values-in-machine-learning-1a6326adf79e
    # 1- We cant just delete them because we need consistent timestamps
    # 2- If too many empty rows exists we should discard them
    # 3- Replacing missing data with mean/median
    # 3.1- This does not cover the covariance between features
    
    #df['PM10 ( µg/m3 )'] = df['PM10 ( µg/m3 )'].replace (-1.0, df[df != -1]['PM10 ( µg/m3 )'].median ())
    #df['SO2 ( µg/m3 )'] = df['SO2 ( µg/m3 )'].replace (-1.0, df[df != -1]['SO2 ( µg/m3 )'].median ())
    #df['CO ( µg/m3 )'] = df['CO ( µg/m3 )'].replace (-1.0, df[df != -1]['CO ( µg/m3 )'].median ())
    #df['NO2 ( µg/m3 )'] = df['NO2 ( µg/m3 )'].replace (-1.0, df[df != -1]['NO2 ( µg/m3 )'].median ())
    #df['NOX ( µg/m3 )'] = df['NOX ( µg/m3 )'].replace (-1.0, df[df != -1]['NOX ( µg/m3 )'].median ())
    #df['O3 ( µg/m3 )'] = df['O3 ( µg/m3 )'].replace (-1.0, df[df != -1]['O3 ( µg/m3 )'].median ())
    #df['PM 2.5 ( µg/m3 )'] = df['PM 2.5 ( µg/m3 )'].replace (-1.0, df[df != -1]['PM 2.5 ( µg/m3 )'].median ())
    if not(df['PM10 ( µg/m3 )'][df['PM10 ( µg/m3 )']!=-1].count()>0):df['PM10 ( µg/m3 )']=0
    if not(df['SO2 ( µg/m3 )'][df['SO2 ( µg/m3 )']!=-1].count()>0):df['SO2 ( µg/m3 )']=0
    if not(df['CO ( µg/m3 )'][df['CO ( µg/m3 )']!=-1].count()>0):df['CO ( µg/m3 )']=0
    if not(df['NO2 ( µg/m3 )'][df['NO2 ( µg/m3 )']!=-1].count()>0):df['NO2 ( µg/m3 )']=0
    if not(df['NOX ( µg/m3 )'][df['NOX ( µg/m3 )']!=-1].count()>0):df['NOX ( µg/m3 )']=0
    if not(df['O3 ( µg/m3 )'][df['O3 ( µg/m3 )']!=-1].count()>0):df['O3 ( µg/m3 )']=0
    if not(df['PM 2.5 ( µg/m3 )'][df['PM 2.5 ( µg/m3 )']!=-1].count()>0):df['PM 2.5 ( µg/m3 )']=0

    #if not(df['PM10 ( µg/m3 )'].notnull().values.any()): df['PM10 ( µg/m3 )']=0
    #if not(df['SO2 ( µg/m3 )'].notnull().values.any()): df['SO2 ( µg/m3 )']=0
    #if not(df['CO ( µg/m3 )'].notnull().values.any()): df['CO ( µg/m3 )']=0
    #if not(df['NO2 ( µg/m3 )'].notnull().values.any()): df['NO2 ( µg/m3 )']=0
    #if not(df['NOX ( µg/m3 )'].notnull().values.any()): df['NOX ( µg/m3 )']=0
    #if not(df['O3 ( µg/m3 )'].notnull().values.any()): df['O3 ( µg/m3 )']=0
    #if not(df['PM 2.5 ( µg/m3 )'].notnull().values.any()): df['PM 2.5 ( µg/m3 )']=0

    df = fillWithSameHourValue(df, 'PM10 ( µg/m3 )')
    df = fillWithSameHourValue(df, 'SO2 ( µg/m3 )')
    df = fillWithSameHourValue(df, 'CO ( µg/m3 )')
    df = fillWithSameHourValue(df, 'NO2 ( µg/m3 )')
    df = fillWithSameHourValue(df, 'NOX ( µg/m3 )')
    df = fillWithSameHourValue(df, 'O3 ( µg/m3 )')
    df = fillWithSameHourValue(df, 'PM 2.5 ( µg/m3 )')
    return df

def fillWithSameHourValue(df, pollutantColumnName):
    for x in range (df[pollutantColumnName].shape[0]):
        if(df[pollutantColumnName].iloc[x] == -1 and x < 24):
            df[pollutantColumnName].iloc[x] = findValue(df[pollutantColumnName], x)
        if(df[pollutantColumnName].iloc[x] == -1 and x >= 24):
            df[pollutantColumnName].iloc[x] = df[pollutantColumnName].iloc[x-24]
        
    return df

def findValue(df, i):
    if(df.shape[0]<=i):
        return 0

    if(df.iloc[i] == -1):
        i = i+24
        return findValue(df, i)
    else:
        return df.iloc[i] 

def preprocessingx(df):
    df = fixValues(df)
    df = fillEmptyRows(df)
    return df

In [9]:
# making preprocessing (fixing values, filling empty rows)
dfAksaray = preprocessing(dfAksaray)
dfBesiktas = preprocessing(dfBesiktas)
dfBuyukada = preprocessing(dfBuyukada)
dfCatladıkapı = preprocessing(dfCatladıkapı)
dfEsenler = preprocessing(dfEsenler)
dfKadıkoy = preprocessing(dfKadıkoy)
dfKandilli = preprocessing(dfKandilli)
dfKartal = preprocessing(dfKartal)
dfMecidiyekoy = preprocessing(dfMecidiyekoy)
dfUmraniye = preprocessing(dfUmraniye)

/Users/deneme/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [22]:
# Initialize data to Dicts of series.  
lastNItems = 20000
columnName = 'PM10 ( µg/m3 )'
excelName = 'datasets/training/PM10.xlsx'

d = {
    'Aksaray' : dfAksaray[columnName][-lastNItems:].tolist(),  
    'Besiktas' : dfBesiktas[columnName][-lastNItems:].tolist(),
    'Buyukada' : dfBuyukada[columnName][-lastNItems:].tolist(), 
    'Catladıkapı' : dfCatladıkapı[columnName][-lastNItems:].tolist(),
    'Esenler' : dfEsenler[columnName][-lastNItems:].tolist(), 
    'Kadıkoy' : dfKadıkoy[columnName][-lastNItems:].tolist(),
    'Kandilli' : dfKandilli[columnName][-lastNItems:].tolist(), 
    'Kartal'    : dfKartal[columnName][-lastNItems:].tolist(),
    'Mecidiyekoy' : dfMecidiyekoy[columnName][-lastNItems:].tolist(), 
    'Umraniye'    : dfUmraniye[columnName][-lastNItems:].tolist()
    
}  
  
# creates Dataframe.  
dframe = pd.DataFrame(d, columns = ['Aksaray', 'Besiktas','Buyukada','Catladıkapı','Esenler','Kadıkoy','Kandilli','Kartal','Mecidiyekoy','Umraniye']) 
  
# print the data.  
print(dframe) 

# export to excel
dframe.to_excel(excelName) 

         Aksaray   Besiktas   Buyukada  Catladıkapı    Esenler    Kadıkoy  \
0      42.200001  37.599998   9.600000    24.100000  28.100000  21.400000   
1      41.799999  42.000000  12.000000    27.299999  34.200001  24.200001   
2      44.200001  39.000000  12.800000    40.200001  33.099998  18.799999   
3      41.700001  37.599998  13.800000    41.599998  33.799999  18.000000   
4      49.700001  48.500000  16.400000    40.900002  36.000000  19.500000   
...          ...        ...        ...          ...        ...        ...   
19995  23.600000  11.100000  13.000000    13.800000  12.200000  14.400000   
19996  25.900000  17.400000   8.700000    17.200001  18.200001  17.600000   
19997  27.600000  30.000000  13.300000    14.600000  18.299999  24.000000   
19998  19.700001  18.600000  30.600000    14.800000  22.000000  30.799999   
19999  27.100000  13.200000  26.200001    16.400000  31.299999  41.200001   

       Kandilli     Kartal  Mecidiyekoy   Umraniye  
0           6.8  46.59

## Sensor Locations

In [17]:
from math import cos, asin, sqrt, pi

# Distance between two coordinates taken from
# https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula
def coord_distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...

In [18]:
sensor_l = pd.read_excel(
     os.path.join("datasets/adjacency/sensor_locations.xlsx"),
     engine='openpyxl',
     parse_dates=True,
)
n_sensors = sensor_l.shape[0]
print (sensor_l)
print (sensor_l.shape)

          İlçe    Enlem   Boylam
0      aksaray  41.0244  29.0997
1     besiktas  41.0520  29.0094
2     buyukada  40.8521  29.1180
3  catladıkapı  41.0023  28.9751
4      esenler  41.0368  28.8880
5      kadıkoy  40.9908  29.0333
6     kandilli  41.0624  29.0582
7       kartal  40.9110  29.1830
8  mecidiyekoy  41.0659  28.9944
9     umraniye  41.0126  29.1618
(10, 3)


In [19]:
# Preparing adjacency matrix for the sensor nodes
distance_matrix = np.zeros ((n_sensors, n_sensors))

# Create distance matrix from each sensor to other ones
# For each sensor
ix = 0
for sensor in sensor_l['İlçe']:
    s_lat = sensor_l[sensor_l['İlçe'] == sensor].values[0][1]
    s_lon = sensor_l[sensor_l['İlçe'] == sensor].values[0][2]
    # Look at other sensors
    o_ix = 0
    for o_sensor in sensor_l['İlçe']:
        o_lat = sensor_l[sensor_l['İlçe'] == o_sensor].values[0][1] 
        o_lon = sensor_l[sensor_l['İlçe'] == o_sensor].values[0][2] 
        # Calculate the distance
        distance = coord_distance (s_lat, s_lon, o_lat, o_lon)
        # Update the distance matrix
        distance_matrix [ix][o_ix] = distance
        #
        o_ix += 1
    ix += 1
    
# Take the absolute of the difference between all values and the max value
# Then divide it by the max value to get the weighted adjacency matrix
# Add 1 to the max value so that the weights will never be lost
max_val = np.max (distance_matrix) + 1
distance_matrix = (max_val - distance_matrix)/max_val
distance_matrix

array([[1.        , 0.72246006, 0.34720973, 0.63525996, 0.39507831,
        0.77216223, 0.81407726, 0.51027434, 0.66159891, 0.81752721],
       [0.72246006, 1.        , 0.18399078, 0.78839024, 0.64948686,
        0.75905202, 0.85558757, 0.27301488, 0.93232307, 0.54105276],
       [0.34720973, 0.18399078, 1.        , 0.30142131, 0.04239443,
        0.42311837, 0.18768574, 0.71030258, 0.11898075, 0.38111595],
       [0.63525996, 0.78839024, 0.30142131, 1.        , 0.71969965,
        0.828519  , 0.67203794, 0.31409166, 0.75359924, 0.46650847],
       [0.39507831, 0.64948686, 0.04239443, 0.71969965, 1.        ,
        0.5509857 , 0.50571456, 0.03396331, 0.67766267, 0.21457354],
       [0.77216223, 0.75905202, 0.42311837, 0.828519  , 0.5509857 ,
        1.        , 0.72044807, 0.47736705, 0.6954975 , 0.62461969],
       [0.81407726, 0.85558757, 0.18768574, 0.67203794, 0.50571456,
        0.72044807, 1.        , 0.32657829, 0.8178543 , 0.65005373],
       [0.51027434, 0.27301488, 0.7103025

In [20]:
# export to excel
workbook = xlsxwriter.Workbook('datasets/training/sensor_dist_adj.xlsx')
worksheet = workbook.add_worksheet()

row = 0

for col, data in enumerate(distance_matrix):
    worksheet.write_column(row, col, data)

workbook.close()

## Population

In [21]:
sensor_l = pd.read_excel(
     os.path.join("datasets/adjacency/population.xlsx"),
     engine='openpyxl',
     parse_dates=True,
)
n_sensors = sensor_l.shape[0]
print (sensor_l)
print (sensor_l.shape)

     Yıl           İlçe  İlçe Nüfusu  Erkek Nüfusu  Kadın Nüfusu  \
0   2020         Adalar        16033          8358          7675   
1   2020     Arnavutköy       296709        152668        144041   
2   2020       Ataşehir       422594        207697        214897   
3   2020        Avcılar       436897        219428        217469   
4   2020       Bağcılar       737206        374475        362731   
5   2020   Bahçelievler       592371        298341        294030   
6   2020       Bakırköy       226229        105741        120488   
7   2020     Başakşehir       469924        236589        233335   
8   2020     Bayrampaşa       269950        135664        134286   
9   2020       Beşiktaş       176513         80715         95798   
10  2020         Beykoz       246110        122425        123685   
11  2020     Beylikdüzü       365572        178615        186957   
12  2020        Beyoğlu       226396        116771        109625   
13  2020   Büyükçekmece       257362        1274